In [1]:
import query_ris

from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, GeoJSON, Marker, MarkerCluster, AwesomeIcon, LegendControl
from ipydatetime import DatetimePicker
from IPython.core.display import display
import ipyleaflet
from ipywidgets import widgets, Layout
from datetime import datetime, timezone
import dateutil
import dateutil.parser
import pytz

import plotly.io as pio

%load_ext ipycache

import warnings
warnings.filterwarnings('ignore')

no dataset global attributes found for:  https://thredds.met.no/thredds/dodsC/met.no/observations/stations/SN99710.nc
no dataset global attributes found for:  https://thredds.met.no/thredds/dodsC/met.no/observations/stations/SN99710.nc


/home/dboulang/anaconda3/lib/python3.8/site-packages/IPython/config.py:12: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  warn("The `IPython.config` package has been deprecated since IPython 4.0. "
/home/dboulang/anaconda3/lib/python3.8/site-packages/ipycache.py:17: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils.traitlets import Unicode


In [5]:
def get_start_date():
    return datetime(2000, 1, 1, 0, 0, tzinfo=pytz.utc)

def get_end_date():
    return datetime.now(timezone.utc)

# Launch search
def on_search_clicked(b):
    with out:
        out.clear_output()
        start = datetime_picker_start.value.strftime("%Y-%m-%dT%H:%M:%S")
        end = datetime_picker_end.value.strftime("%Y-%m-%dT%H:%M:%S")
        selected_platforms = get_selected_platforms(platforms, bbox)
        print("QUERY\nSelected geographical area: " + str(bbox) + "\nSelection period: " + str(start) + " and " + str(end)+ "\nSelected variables: " + str(list_variables.value) + "\nSelected platforms: " + str(selected_platforms))
        datasets=query_ris.query_datasets(start, end, list_variables.value, bbox)
        
        #import query_icos
        #datasets=query_icos.query_datasets(['Pressure (surface)'], ['2018-01-01T03:00:00','2021-12-31T24:00:00'],[10, 40, 23, 60])
        #print(datasets)
        
        datasets2 = []
        for ri in datasets.keys():
            for endpoint in datasets[ri]:
                if endpoint.startswith("http"):
                     datasets2.append(endpoint)
        list_datasets.options=datasets2
        #list_datasets = selection_datasets_list(datasets)
        datasets_panel = widgets.HBox([list_datasets])
        datasets_panel.layout.visibility = 'visible'

# Clear output panel
def mk_clear_button(target, action_name):
    button = widgets.Button(description=action_name.replace('_', ' '))
    action = getattr(target, action_name)
    button.on_click(lambda *a: action())
    return button

def mk_search_button():
    button = widgets.Button(description="Search")
    button.on_click(on_search_clicked)
    return button

# Display coordinates
def handle_interaction(**kwargs):
    if kwargs.get('type') == 'mousemove':
        coords = kwargs.get('coordinates')
        lat_label.value = str(coords[0]) 
        lon_label.value = str(coords[1])

def handle_click(**kwargs):
    with out:
        print(kwargs)
        print(type(kwargs))
        # kwargs['feature']['properties'][foobar] = 10 
        
def on_draw_handler(draw, action, geo_json):
    with out:
        out.clear_output()
        for i in map_panel.layers:
            if type(i) == ipyleaflet.GeoJSON:
                map_panel.remove_layer(i)
        bounds = geo_json['geometry']['coordinates'][0]
        ll = bounds[0]
        ur = bounds[2]
        corners = [ll, ur]
        global bbox
        bbox = [item for sublist in corners for item in sublist]
        searchbox = GeoJSON(data=geo_json)
        searchbox.on_click(handle_click)
        map_panel.add_layer(searchbox)
        draw.clear_rectangles()
        
def selection_variables_list(values):
    return widgets.SelectMultiple(options=values,
    rows=10,
    description='Variables',
    disabled=False,
    layout=Layout(width='800px',)
)    

def selection_datasets_list(values):
    return widgets.SelectMultiple(options=values,
    rows=30,
    description='Datasets',
    disabled=False,
    layout=Layout(width='800px',)
)

ri_colors= { 'actris': '#37a7da', 'iagos': '#72af26', 'icos': '#d03c29', 'sios': '#f3952f' }
ri_icons = { 'actris': AwesomeIcon(
    name='map-marker',
    marker_color='blue',
    icon_color='black',
    spin=False
), 'iagos': AwesomeIcon(
    name='plane',
    marker_color='green',
    icon_color='black',
    spin=False
), 'icos': AwesomeIcon(
    name='map-marker',
    marker_color='red',
    icon_color='black',
    spin=False
), 'sios': AwesomeIcon(
    name='map-marker',
    marker_color='orange',
    icon_color='black',
    spin=False
) }

def test_within(bbox, x, y) :
    if (x > bbox[0] and x < bbox[2] and y > bbox[1] and y < bbox[3]) :
        return True
    else:
        return False

def get_selected_platforms(platforms, bbox):
    selection = {}
    for ri in platforms.keys():
        sel = []
        for platform in platforms[ri]:
            if test_within(bbox, float(platform['longitude']), float(platform['latitude'])):
                sel.append(platform)
        if len(sel) > 0:
            selection[ri] = sel
    return selection

def display_platforms(platforms, m):
    markers=[]
    for ri in ri_icons.keys():
        for platform in platforms[ri]:
            center = (platform['latitude'], platform['longitude'])
            marker = Marker(icon=ri_icons[ri], location=center, draggable=False, title=(platform['short_name'] + " - " + ri.upper()))
            markers.append(marker)
    marker_cluster = MarkerCluster(markers=markers)
    m.add_layer(marker_cluster);

In [6]:
%%cache mycache2.pkl platforms variables
platforms = query_ris.load_platforms()
variables = query_ris.load_variables()

[Skipped the cell's code and loaded variables platforms, variables from file '/home/dboulang/eclipse-workspace/envri-wp8-demonstrator/src/mycache2.pkl'.]
... loading actris platforms
... loading iagos platforms
... loading icos platforms
... loading sios platforms
all the platforms are loaded
... loading actris variables
... loading iagos variables
... loading icos variables
... loading sios variables
all the variables are loaded


In [8]:
# Temporal extent
datetime_picker_start = DatetimePicker(value=get_start_date())
datetime_picker_end = DatetimePicker(value=get_end_date())

# Output panel
out = widgets.Output(layout=widgets.Layout(width='50%', 
                               height='400px', 
                               overflow_y='scroll',
                               border='1px solid black'))
clear_output = mk_clear_button(out, 'clear_output')
search = mk_search_button()

# Map panel
map_panel = ipyleaflet.Map(layers=(basemap_to_tiles(basemaps.Stamen.Terrain), ), center=(49, 11), zoom=4, attribution_control=False)
map_panel.layout.height = '600px'
controls = ipyleaflet.LayersControl(position='topright')
lat_label = widgets.Label()
lon_label = widgets.Label()
map_panel.add_control(controls)
map_panel.on_interaction(handle_interaction)
global bbox
bbox = [-180, -90, 180, 90]

list_variables = selection_variables_list(query_ris.get_ECV(variables))
list_datasets = selection_datasets_list(['none'])

#Draw panel for selection on map
draw = ipyleaflet.DrawControl(
    edit=True,
    remove=True,
    circlemarker={},
    marker={},
    circle={},    
    polyline={},
    polygon = {},
    rectangle={'shapeOptions': {}})
draw.on_draw(on_draw_handler)
map_panel.add_control(draw)

legend = LegendControl({'ACTRIS': ri_colors['actris'], 'IAGOS': ri_colors['iagos'], 'ICOS': ri_colors['icos'], 'SIOS': ri_colors['sios']}, name="Legend", position="bottomright")
map_panel.add_control(legend)

display_platforms(platforms, map_panel)

datasets_panel = widgets.HBox([list_datasets])
#datasets_panel.layout.visibility = 'hidden'

dashboard = widgets.VBox([                      
                    widgets.HBox([list_variables, widgets.VBox([widgets.HBox([datetime_picker_start, datetime_picker_end]), clear_output, search])]), 
                    widgets.HBox([map_panel, out]), 
                    datasets_panel
])

dashboard